# MNL推定を行うためのコード
準備したデータを用いてBiogemeで交通手段選択モデルを推定します．

In [6]:
import numpy as np
import pandas as pd

import biogeme.database as db
from biogeme.expressions import Variable

import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta


## データの読み込み&加工
準備したデータをpandasで読み込んで以下の加工を行います: 
1. 着目する交通手段だけに限定したデータにする．
2. Biogemeでは，ヘッダー行以外で文字(string)は読み込めないので，文字を含む列を削除する．
2. Biogemeでは，NaNも読み込めないので，NaNを含む行は削除する．
2. num列に記載されている，変数の組み合わせが同一であるサンプル数の分だけ，行を複製する(非集計データにする)．

In [8]:
df = pd.read_csv("../data/data.csv", encoding='shift_jis')

# 着目する交通手段に限定（鉄道・バス・乗用車等）
df = df[(df['mode_name'] == "幹線バス") | (df['mode_name'] == "鉄道") | (df['mode_name'] == "乗用車等")].reset_index(drop=True).copy()

# 不要な列を削除（文字列列・使用しないLOS変数）
df = df.drop(columns=['O_name', 'D_name', 'purpose_name', 'mode_name', 'sex_name', 'ship_time', 'ship_cost', 'air_time', 'air_cost']).copy()

# 欠損値を含む行を削除
df = df.dropna().reset_index(drop=True).copy()

# トリップ数に応じてデータを展開（各行を'num'回繰り返す）
df = df.loc[df.index.repeat(df['num'])].reset_index(drop=True).copy() 
display(df)

,O_code,D_code,purpose_code,mode_code,sex_code,age_code,num,car_time,bus_time,rail_time,car_cost,bus_cost,rail_cost
0,1,48,1,2,1.0,30.0,2.0,304.3508,515.8,397.3,2900.0,5450.0,13310.0
1,1,48,1,2,1.0,30.0,2.0,304.3508,515.8,397.3,2900.0,5450.0,13310.0
2,1,48,1,2,1.0,40.0,2.0,304.3508,515.8,397.3,2900.0,5450.0,13310.0
3,1,48,1,2,1.0,40.0,2.0,304.3508,515.8,397.3,2900.0,5450.0,13310.0
4,1,48,1,2,2.0,30.0,3.0,304.3508,515.8,397.3,2900.0,5450.0,13310.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216679,46,45,0,5,2.0,70.0,108.0,154.2540,276.9,208.4,4110.0,2950.0,4390.0
2216680,46,45,0,5,2.0,70.0,108.0,154.2540,276.9,208.4,4110.0,2950.0,4390.0
2216681,46,45,0,5,2.0,70.0,108.0,154.2540,276.9,208.4,4110.0,2950.0,4390.0
2216682,46,45,0,5,2.0,70.0,108.0,154.2540,276.9,208.4,4110.0,2950.0,4390.0


## Biogemeによる推定
### データの読み込み

In [10]:
# Biogeme形式のデータに変換
database = db.Database ('junryudou', df)

### 各列の数値の変数への変換
推定に用いる変数を定義します．
データベース内の各列をモデルの変数とするために，以下のように記述します． 

In [12]:
mode = Variable('mode_code')  # 実際に選択した交通手段のコード
car_time = Variable('car_time')  # 自動車の所要時間
bus_time = Variable('bus_time')  # 幹線バスの所要時間
rail_time = Variable('rail_time')  # 鉄道の所要時間
car_cost = Variable('car_cost')  # 自動車の所要費用
bus_cost = Variable('bus_cost')  # 幹線バスの所要費用
rail_cost = Variable('rail_cost')  # 鉄道の所要費用

変数のスケーリングを行う場合は，つぎのように，変数を再定義します．

In [14]:
car_time_scaled = car_time / 60
car_cost_scaled = car_cost / 10000
rail_time_scaled = rail_time / 60
rail_cost_scaled = rail_cost / 10000
bus_time_scaled = bus_time / 60
bus_cost_scaled = bus_cost / 10000

## パラメータの定義

関数Betaを用いて，パラメータを定義します．
関数Betaでは，パラメータ名，初期値，下限値，上限値，推定の要否を指定します．

In [16]:
# parameter = Beta('name', value, lowerBound, upperBound, status)
# status: 0=推定対象, 1=固定値

ASC_car = Beta('ASC_car', 0, None, None, 0)    # 乗用車の選択肢固有係数
ASC_rail = Beta('ASC_rail', 0, None, None, 0)  # 鉄道の選択肢固有係数
ASC_bus = Beta('ASC_bus', 0, None, None, 1)    # バスの選択肢固有定数（基準：固定）
B_time = Beta('B_time', 0, None, None, 0)      # 所要時間の選択肢共通係数
B_cost = Beta('B_cost', 0, None, None, 0)      # 費用の選択肢共通係数

## 効用関数の定義
以上で定義した変数とパラメータを使って，効用関数を記述します．
今回は次の式のように各選択肢の効用が各選択肢の所要時間と所要費用で表されるものとします．
また，パラメータは選択肢共通のものとします．

In [20]:
V_car = ASC_car + B_time * car_time_scaled + B_cost * car_cost_scaled
V_rail = ASC_rail + B_time * rail_time_scaled + B_cost * rail_cost_scaled
V_bus = ASC_bus + B_time * bus_time_scaled + B_cost * bus_cost_scaled

## 選択結果との対応付け・選択可能性の設定
各効用関数をデータ上の選択肢番号と対応付けます．
今回は，mode列に自動車が5，鉄道が2，幹線バスが4とされているので，各々の効用関数と対応付けます．
これには，辞書型の変数`V`を用います．

In [22]:
# mode_codeと効用関数の対応付け
V = {5: V_car, 2: V_rail, 4: V_bus}

また，選択可能性についても各選択肢番号に対して個別に設定します．
選択可能性とは，例えば，運転免許を持っていない人は，自動車が選択肢になり得ないといった，そもそも選択肢になり得ない状況を表現するためのものです．
今回はすべてのサンプルがすべての選択肢を選択できるものとします．

In [24]:
# 選択可能性(1: 選択可, 0: 選択不可)
av = {5: 1, 2: 1, 4: 1}

## モデルの定義と推定
推定のためにモデルを定義します．
`models.loglogit(V, av, mode)`では，ロジットモデルの選択確率の対数を返します．
引数には，上記で定義した効用関数の辞書，選択可能性の辞書，選択結果の変数を渡します．

In [26]:
# 選択確率の対数を計算
logprob = models.loglogit(V, av, mode)

続いて，BIOGEMEオブジェクトを準備します．
biogeme公式の説明では，この変数にbiogemeという名前を設定するのは避け，たとえば，the_biogemeのような名前とすることが推奨されています．
`database`は，「データの読み込み・加工」で定義した推定用のデータを格納した変数です．

In [28]:
# BIOGEMEオブジェクトの準備
the_biogeme = bio.BIOGEME(database, logprob)

File biogeme.toml has been created


次のようにモデル名を定義します．
ここで定義した名前が，推定結果を記載したレポートファイルのファイル名となります．

In [29]:
# モデル名の定義
the_biogeme.model_name = 'mode_MNL'

各係数が0である場合の尤度はモデルの適合度評価に用いる重要な指標です．
次のような記述をすることで計算することができます．

In [32]:
# ヌルモデルの対数尤度を計算
the_biogeme.calculate_null_loglikelihood(av)

-2435276.28249398

次のように記述することで，モデルの推定を実行します．

In [34]:
# モデル推定の実行
MNL_results = the_biogeme.estimate()

## 推定結果の出力
モデルの推定結果を表示します．
こちらのコードで表示される推定結果のほかに，ディレクトリ内に推定結果をまとめたレポートファイルが出力されます．

In [35]:
print(MNL_results.short_summary())

Results for model mode_MNL
Nbr of parameters:		4
Sample size:			2216684
Excluded data:			0
Null log likelihood:		-2435276
Final log likelihood:		-1111394
Likelihood ratio test (null):		2647765
Rho square (null):			0.544
Rho bar square (null):			0.544
Akaike Information Criterion:	2222795
Bayesian Information Criterion:	2222846



## t値の計算
推定結果のレポートファイルで，推定されたパラメータのの$t$値は，ロバスト推定値のみ記載されています．
そこで，非ロバストの$t$値も出力されるようにします．
まずは，パラメータの推定値とその分散共分散行列を取得します．

In [36]:
# ライブラリのインポート
from biogeme.results_processing.estimation_results import EstimationResults
from biogeme.results_processing.variance_covariance import EstimateVarianceCovariance

# MNL_results が RawEstimationResults の場合に EstimationResults でラップ
res = MNL_results if isinstance(MNL_results, EstimationResults) else EstimationResults(MNL_results)

# 推定値（β）
betas: dict[str, float] = res.get_beta_values()

# 通常（Rao–Cramer）の分散共分散行列を取得
vc = res.get_variance_covariance_matrix(
    variance_covariance_type=EstimateVarianceCovariance.RAO_CRAMER # ROBUSTを指定すると，ロバスト値を得る
)

非ロバストの$t$値と$p$値を計算し，表敬式で出力します．

In [65]:
# 標準誤差と t 値を計算
names = list(betas.keys())
values = np.array([betas[n] for n in names], dtype=float)
se = np.sqrt(np.diag(np.asarray(vc, dtype=float)))
t_vals = values / se

# p 値
from math import erf, sqrt
def p_from_t(t):
    Phi = 0.5 * (1.0 + erf(abs(t) / sqrt(2.0)))  # 一部erfを用いて標準正規分布の累積分布関数を設定
    return 2.0 * (1.0 - Phi)  # 両側検定

p_vals = [p_from_t(t) for t in t_vals]

# 表として出力
names = list(betas.keys())  # dict型の変数betasに対して，keysメソッドを実行
df_nonrobust = pd.DataFrame(
    {"Value": values,
     "Std err. (non-robust)": se,
     "t-test (non-robust)": t_vals,
     "p-value (non-robust)": p_vals},
    index=names
)

display(df_nonrobust)

,Value,Std err. (non-robust),t-test (non-robust),p-value (non-robust)
ASC_car,2.465231,0.005218,472.439971,0.0
B_time,-0.930564,0.001487,-625.897156,0.0
B_cost,-1.473249,0.012004,-122.729771,0.0
ASC_rail,0.665556,0.006766,98.373092,0.0


## 的中率の計算
各パラメータの推定値を取得し，変数に代入します．

In [68]:
display(betas)

{'ASC_car': 2.4652311788350993,
 'B_time': -0.9305637878132774,
 'B_cost': -1.4732485578910623,
 'ASC_rail': 0.6655562347518086}

In [72]:
# パラメータの推定値の取得と変数への代入
res_ASC_car = betas['ASC_car']
res_ASC_rail = betas['ASC_rail']
res_B_time = betas['B_time']
res_B_cost = betas['B_cost']

パラメータの推定値を用いて，効用を計算します．

In [74]:
# 効用の計算
df['pred_V_car'] = res_ASC_car + res_B_time * df['car_time'] / 60 + res_B_cost * df['car_cost'] / 10000
df['pred_V_rail'] = res_ASC_rail + res_B_time * df['rail_time'] / 60 + res_B_cost * df['rail_cost'] / 10000
df['pred_V_bus'] = res_B_time * df['bus_time'] / 60 + res_B_cost * df['bus_cost'] / 10000

さらに，ロジットモデルの式を用いて，各選択肢の選択確率を計算します．

In [77]:
# 選択確率の分母
deno = np.exp(df['pred_V_car']) + np.exp(df['pred_V_rail']) + np.exp(df['pred_V_bus'])

# 各選択肢の選択確率
df['pred_P_car'] = np.exp(df['pred_V_car']) / deno
df['pred_P_rail'] = np.exp(df['pred_V_rail']) / deno
df['pred_P_bus'] = np.exp(df['pred_V_bus']) / deno

計算された各選択肢の選択確率のうち，最大であった選択肢が選ばれると予測します．

In [80]:
# 選択確率が最大であった選択肢に選択肢番号を対応付け
cols = ['pred_P_car', 'pred_P_rail', 'pred_P_bus']
code_map = {'pred_P_car': 5, 'pred_P_rail': 2, 'pred_P_bus': 4}
df['pred_mode'] = df[cols].idxmax(axis=1).map(code_map).astype('Int64')

実際に選択された選択肢と選択されると予測された選択肢が一致している場合には，的中，そうでない場合は，不的中であるとします．
さらにその判定結果をもとに的中率を計算します．

In [83]:
# 的中と不的中の割り当て
df['judge'] = np.where(df['pred_mode'].astype('Int64').eq(df['mode_code'].astype('Int64')),
                       'correct', 'wrong')
                       
# 的中となったサンプル数を数えて，的中率を計算
print( ((df["judge"] == "correct").sum() + (df["judge"] == "wrong").sum()) == len(df))
acc = (df["judge"] == "correct").sum() / ((df["judge"] == "correct").sum() + (df["judge"] == "wrong").sum()) * 100

print(f"的中率: {acc}%")

True
的中率: 82.66884228875203%
